In [28]:
%matplotlib inline

import matplotlib.pyplot as plt
import scipy

import numpy as np
import pandas as pd
import xarray as xr
import collections
# import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# sns.set(style='darkgrid')

from skdownscale.pointwise_models import BcsdTemperature, BcsdPrecipitation

In [29]:
from skdownscale.pointwise_models import PaddedDOYGrouper

In [30]:
from utils import get_sample_data

In [31]:
# ! pip install probscale

In [32]:
training = get_sample_data('training')
targets = get_sample_data('targets')

In [34]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.23.2.


In [35]:
train_slice = slice('1980-01-01', '1989-12-31')
predict_slice = slice('1990-01-01', '1999-12-31')

# extract training / prediction data
X_train = training[['tmax']][train_slice]
y_train = targets[['tmax']][train_slice]
X_predict = training[['tmax']][predict_slice]
    
model = BcsdTemperature(return_anoms=False).fit(X_train, y_train)

In [36]:
X_train_noleap = X_train[~((X_train.index.month == 2) & (X_train.index.day == 29))]
ts_daysofyear = X_train.index.dayofyear
padded_doy = np.pad(ts_daysofyear, 15, mode="wrap")

In [37]:
len(X_train_noleap[((X_train_noleap.index.month == 2) & (X_train_noleap.index.day == 29))])

0

tests to add to scikit-downscale 

for testing PaddedDOYGrouper

In [38]:
index = pd.date_range(start='1980-01-01', end='1982-12-31')
X = pd.DataFrame({'foo': np.random.random(len(index))}, index=index)
day_groups = PaddedDOYGrouper(X)
u = PaddedDOYGrouper(X_train_noleap[slice('1981-01-01', '1982-12-31')])
doy_group_list = dict(list(u))

day_of_year = 123
days_included = np.arange(day_of_year - 15, day_of_year + 16)
days_included_to_fail = np.arange(day_of_year - 15, day_of_year + 18)
np.testing.assert_array_equal(np.unique(doy_group_list[day_of_year].index.dayofyear), days_included)

In [39]:
bool(doy_group_list)

True

In [40]:
# u = X_train.groupby(PaddedDOYGrouper)
u = PaddedDOYGrouper(X_train_noleap[slice('1980-01-01', '1982-12-31')])
# u = PaddedDOYGrouper(X_train[slice('1980-01-01', '1982-12-31')])
doy_group_list = dict(list(u))

In [41]:
v = np.pad(np.arange(1, 366), 15, mode="wrap")
i = 0
offset = 15
n = 1
total_days = (2 * offset) + 1
first_half = v[i : i + offset]
sec_half = v[n + offset : i + total_days]

In [42]:
first_half

array([351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363,
       364, 365])

In [43]:
year_days = pd.date_range(start='1980-01-01', end='1982-12-31')
print(year_days[81])
doy_group_list[82].index

1980-03-22 00:00:00


DatetimeIndex(['1980-03-07', '1980-03-08', '1980-03-09', '1980-03-10',
               '1980-03-11', '1980-03-12', '1980-03-13', '1980-03-14',
               '1980-03-15', '1980-03-16', '1980-03-17', '1980-03-18',
               '1980-03-19', '1980-03-20', '1980-03-21', '1980-03-22',
               '1980-03-23', '1980-03-24', '1980-03-25', '1980-03-26',
               '1980-03-27', '1980-03-28', '1980-03-29', '1980-03-30',
               '1980-03-31', '1980-04-01', '1980-04-02', '1980-04-03',
               '1980-04-04', '1980-04-05', '1980-04-06', '1981-03-08',
               '1981-03-09', '1981-03-10', '1981-03-11', '1981-03-12',
               '1981-03-13', '1981-03-14', '1981-03-15', '1981-03-16',
               '1981-03-17', '1981-03-18', '1981-03-19', '1981-03-20',
               '1981-03-21', '1981-03-22', '1981-03-23', '1981-03-24',
               '1981-03-25', '1981-03-26', '1981-03-27', '1981-03-28',
               '1981-03-29', '1981-03-30', '1981-03-31', '1981-04-01',
      

In [44]:
# leap day check to add into PaddedDOYGrouper

In [45]:
print(len(X_train_noleap[((X_train_noleap.index.month == 2) & (X_train_noleap.index.day == 29))]))

0


In [46]:
# X_predict.head()

In [47]:
predict_df = pd.DataFrame(index = X_predict.index)

# X_rolling_mean, x_climo, y_climo, Xqm, X_no_shift, X, X_shift, df_bc = model.predict(X_predict)
df_bc = model.predict(X_predict)
predict_df['BCSD'] = df_bc.values

In [48]:
df_bc

,tmax
time,
1990-01-01,4.528703
1990-01-02,-1.584749
1990-01-03,2.848937
1990-01-04,6.687826
1990-01-05,8.296425
...,...
1999-12-27,19.128365
1999-12-28,16.533390
1999-12-29,8.740290


In [20]:
# predict_df['BCSD'].plot()

In [21]:
'''fig, ax = plt.subplots(figsize=(8, 3.5))

time_slice = slice('1990-01-01', '1990-12-31')

targets['tmax'][time_slice].plot(ax=ax, label='target', linestyle="--", c='red', lw=1, alpha=0.75, legend=True, zorder=10)

training['tmax'][time_slice].plot(label='original', linestyle="--", c='k', ax=ax, alpha=0.75, legend=True)

predict_df[time_slice].plot(ax=ax, linestyle="-", c='blue', alpha=0.75, legend=True)

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
_ = ax.set_ylabel('Temperature [C]')'''

'fig, ax = plt.subplots(figsize=(8, 3.5))\n\ntime_slice = slice(\'1990-01-01\', \'1990-12-31\')\n\ntargets[\'tmax\'][time_slice].plot(ax=ax, label=\'target\', linestyle="--", c=\'red\', lw=1, alpha=0.75, legend=True, zorder=10)\n\ntraining[\'tmax\'][time_slice].plot(label=\'original\', linestyle="--", c=\'k\', ax=ax, alpha=0.75, legend=True)\n\npredict_df[time_slice].plot(ax=ax, linestyle="-", c=\'blue\', alpha=0.75, legend=True)\n\nax.legend(loc=\'center left\', bbox_to_anchor=(1, 0.5))\n_ = ax.set_ylabel(\'Temperature [C]\')'

### switch to our NASA-NEX daily implementation

In [22]:
# remove leap days 
training_noleap = training[~((training.index.month == 2) & (training.index.day == 29))]
targets_noleap = targets[~((targets.index.month == 2) & (targets.index.day == 29))]

# extract training / prediction data
X_train_noleap = training_noleap[['tmax']][train_slice]
y_train_noleap = targets_noleap[['tmax']][train_slice]
X_predict_noleap = training_noleap[['tmax']][predict_slice]

In [23]:
model_nasanex = BcsdTemperature(time_grouper='daily_nasa-nex', return_anoms=False).fit(X_train_noleap, 
                                                                                       y_train_noleap)

In [24]:
test_model_nasanex = BcsdTemperature(time_grouper='daily_nasa-nex')

In [25]:
assert issubclass(model_nasanex.time_grouper, PaddedDOYGrouper)

In [26]:
u = np.full((365, 1), np.inf)
u[100] = 5

In [27]:
predict_df_nasanex = pd.DataFrame(index = X_predict_noleap.index)

df_bc_nasanex = model_nasanex.predict(X_predict_noleap)

predict_df_nasanex['BCSD'] = df_bc_nasanex.values

In [ ]:
fig, ax = plt.subplots(figsize=(18, 3.5))

time_slice_compare = slice('1990-01-01', '1991-01-01')

# X_predict_noleap['tmax'][time_slice_compare].plot(ax=ax, color='red', label='GCM')

targets['tmax'][time_slice_compare].plot(ax=ax, color='black', linestyle=':', label='targets/obs')

predict_df['BCSD'][time_slice_compare].plot(ax=ax, color='black', linestyle='-', alpha=.75, label='monthly')
predict_df_nasanex['BCSD'][time_slice_compare].plot(ax=ax, color='blue', linestyle='-', alpha=.75, label='nasa-nex')

plt.legend()

plt.ylabel('Temperature ($^\circ$C)')

In [ ]:
fig, ax = plt.subplots(figsize=(18, 3.5))

'''predict_df['BCSD'].plot(color='black', linestyle=':', alpha=.75, label='Joe')
predict_df_nasanex['BCSD'].plot(color='blue', linestyle=':', alpha=.75, label='me')'''

diff = predict_df_nasanex['BCSD'] - predict_df['BCSD']

'''predict_df['BCSD'][time_slice_compare].plot(color='black', linestyle='-', alpha=.75, label='monthly')
predict_df_nasanex['BCSD'][time_slice_compare].plot(color='blue', linestyle='-', alpha=.75, label='nasa-nex')'''

diff.plot()

plt.legend()

### now test precipitation model

In [ ]:
# extract training / prediction data
X_train = training[['pcp']][train_slice]
y_train = targets[['pcp']][train_slice]
X_predict = training[['pcp']][predict_slice]
    
model = BcsdPrecipitation(return_anoms=False).fit(X_train, y_train)

In [ ]:
predict_df = pd.DataFrame(index = X_predict.index)

df_bc = model.predict(X_predict)
predict_df['BCSD'] = df_bc.values

In [ ]:
# remove leap days 
training_noleap = training[~((training.index.month == 2) & (training.index.day == 29))]
targets_noleap = targets[~((targets.index.month == 2) & (targets.index.day == 29))]

# extract training / prediction data
X_train_noleap = training_noleap[['pcp']][train_slice]
y_train_noleap = targets_noleap[['pcp']][train_slice]
X_predict_noleap = training_noleap[['pcp']][predict_slice]

In [ ]:
model_nasanex = BcsdPrecipitation(time_grouper='daily_nasa-nex', return_anoms=False).fit(X_train_noleap, 
                                                                                       y_train_noleap)

In [ ]:
predict_df_nasanex = pd.DataFrame(index = X_predict_noleap.index)

df_bc_nasanex = model_nasanex.predict(X_predict_noleap)

predict_df_nasanex['BCSD'] = df_bc_nasanex.values

In [ ]:
fig, ax = plt.subplots(figsize=(18, 3.5))

time_slice_compare = slice('1990-01-01', '1991-01-01')

X_predict_noleap['pcp'][time_slice_compare].plot(ax=ax, color='brown', linestyle=':', label='GCM')

targets['pcp'][time_slice_compare].plot(ax=ax, color='black', linestyle=':', label='targets/obs')

predict_df['BCSD'][time_slice_compare].plot(ax=ax, color='red', linestyle='-', alpha=.75, label='monthly')

predict_df_nasanex['BCSD'][time_slice_compare].plot(ax=ax, color='blue', linestyle='-', alpha=.75, label='nasa-nex')

plt.legend()

In [ ]:
# self.time_grouper(df)
doy_groups = PaddedDOYGrouper(x_rolling_mean)

In [ ]:
def DAY_GROUPER(x):
    return x.day
doy_groups_day = X_train_noleap.groupby(DAY_GROUPER)

In [ ]:
# doy_groups.get_group(10).head()
doy_group_list = dict(list(doy_groups))

In [ ]:
# group - climatology.loc[key]
# doy_group_list[100] - x_climo[100]
# doy_group_list[100] - x_climo.loc[100]

In [ ]:
dfs = []
for key, group in doy_groups_day:
    dfs.append(group - x_climo.loc[key])

out = pd.concat(dfs).sort_index()
assert x_rolling_mean.shape == out.shape

In [ ]:
print(x_rolling_mean.shape)
print(len(dfs))
print(out.shape)
dfs[100]

In [ ]:
'''# def MONTH_GROUPER(x):
    return x.month
def _create_groups(self, df):
        """ helper function to create groups by either daily or month,
        depending on whether we are bias correcting daily or monthly data
        """
        if self.timestep == 'monthly':
            return df.groupby(self.time_grouper)
        elif self.timestep== 'daily':
            return self.time_grouper(df)
        else:
            raise TypeError('unexpected time grouper type %s' % self.time_grouper)
# self._x_climo = self._create_groups(X).mean()
month_groups = X_train_noleap.groupby(MONTH_GROUPER)

# self.time_grouper(df)
doy_groups = PaddedDOYGrouper(x_rolling_mean)

doy_training = PaddedDOYGrouper(X_train_noleap)

month_groups.get_group(6).head()

# doy_groups.get_group(10).head()
doy_group_list = dict(list(doy_groups))

doy_group_list_training = dict(list(doy_training))

days_of_year = np.arange(1, 366)
days_of_year_wrapped = np.pad(days_of_year, 15, mode='wrap')
n = 5
offset = 15
i = n - 1
total_days = (2 * offset) + 1

first_half = days_of_year_wrapped[i:i+offset]
# sec_half = days_of_year_wrapped[i+offset:i+total_days]
sec_half = days_of_year_wrapped[n+offset:i+total_days]
all_days = np.concatenate((first_half, np.array([n]), sec_half), axis=0)
'''print(first_half)
print(sec_half)
print(all_days)
print(len(first_half))
print(len(sec_half))
print(len(all_days))'''

assert len(set(all_days)) == total_days, all_days

result = X_train_noleap[X_train_noleap.index.dayofyear.isin(all_days)]'''